In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
# preprocessing from ann ex

df = pd.read_csv("../csv/customer_churn.csv")
df.drop(["customerID"], axis="columns", inplace=True)

df1 = df[df.TotalCharges != ' ']
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

df1.replace('No phone service', 'No', inplace=True)
df1.replace('No internet service', 'No', inplace=True)

yes_and_no_cols = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                   'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']

for col in yes_and_no_cols:
    df1[col].replace({"Yes" : 1, "No" : 0}, inplace=True)
    df1[col] = df1[col].astype(np.uint8)

df1.replace({"Male" : 1, "Female" : 0}, inplace=True)
df1["gender"] = df1["gender"].astype(np.uint8)

df2 = pd.get_dummies(data=df1, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True)

bool_cols = ['InternetService_No', 'InternetService_Fiber optic', 'Contract_Two year',
             'Contract_One year','PaymentMethod_Mailed check','PaymentMethod_Credit card (automatic)',
             'PaymentMethod_Electronic check']

for col in bool_cols:
    df2[col] = df2[col].astype(np.uint8)

scaler = MinMaxScaler()
cols_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']
df2[cols_to_scale] = scaler.fit_transform(df2[cols_to_scale])

In [3]:
def ANN(X_train: np.ndarray, X_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray, epochs: int = 100) -> np.ndarray:
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(26, input_dim=X_train.shape[1], activation='relu'),
        tf.keras.layers.Dense(13, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

    model.fit(X_train, y_train, epochs=epochs)

    try:
        print(model.evaluate())

    except:
        print('No evaluation')

    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)

    print('Classification report: \n',classification_report(y_test, y_pred))

    return y_pred

# Unbalaced dataset solutions

### 1. Undersampling

In [4]:
count_class_0, count_class_1 = df1.Churn.value_counts()

df2_zeroes = df2[df2['Churn'] == 0]
df2_ones = df2[df2['Churn'] == 1]

df2_zeroes = df2_zeroes.sample(min(count_class_0, count_class_1))
df2_ones = df2_ones.sample(min(count_class_0, count_class_1))

df2_under = pd.concat([df2_ones, df2_zeroes], axis=0)
print(df2_under.Churn.value_counts())

Churn
1    1869
0    1869
Name: count, dtype: int64


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df2_under.drop('Churn', axis='columns'), df2_under['Churn'], test_size=0.3, stratify=df2_under['Churn'])

In [6]:
y_preds = ANN(X_train, X_test, y_train, y_test)

Epoch 1/100
82/82 [==============================] - 2s 1ms/step - loss: 0.6411 - accuracy: 0.6449
Epoch 2/100
82/82 [==============================] - 0s 1ms/step - loss: 0.5559 - accuracy: 0.7385
Epoch 3/100
82/82 [==============================] - 0s 1ms/step - loss: 0.5161 - accuracy: 0.7489
Epoch 4/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4994 - accuracy: 0.7576
Epoch 5/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4900 - accuracy: 0.7611
Epoch 6/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4845 - accuracy: 0.7603
Epoch 7/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4841 - accuracy: 0.7626
Epoch 8/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4796 - accuracy: 0.7645
Epoch 9/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4760 - accuracy: 0.7653
Epoch 10/100
82/82 [==============================] - 0s 1ms/step - loss: 0.4717 - accuracy: 0.7634
Epoch 11/

### 2. Oversampling

In [7]:
count_class_0, count_class_1 = df1.Churn.value_counts()

df2_zeroes = df2[df2['Churn'] == 0]
df2_ones = df2[df2['Churn'] == 1]

df2_zeroes = df2_zeroes.sample(max(count_class_0, count_class_1), replace=True)
df2_ones = df2_ones.sample(max(count_class_0, count_class_1), replace=True)

print(df2_zeroes.shape, df2_ones.shape)

df2_over = pd.concat([df2_ones, df2_zeroes], axis=0)
print(df2_over.Churn.value_counts())

(5163, 24) (5163, 24)
Churn
1    5163
0    5163
Name: count, dtype: int64


In [8]:
X_train, X_test, y_train, y_test = train_test_split(df2_over.drop('Churn', axis='columns'), df2_over['Churn'], test_size=0.3)
y_pred = ANN(X_train, X_test, y_train, y_test)

Epoch 1/100
226/226 [==============================] - 1s 1ms/step - loss: 0.5677 - accuracy: 0.7110
Epoch 2/100
226/226 [==============================] - 0s 1ms/step - loss: 0.5014 - accuracy: 0.7529
Epoch 3/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4918 - accuracy: 0.7623
Epoch 4/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4863 - accuracy: 0.7659
Epoch 5/100
226/226 [==============================] - 0s 977us/step - loss: 0.4837 - accuracy: 0.7681
Epoch 6/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4800 - accuracy: 0.7678
Epoch 7/100
226/226 [==============================] - 0s 907us/step - loss: 0.4770 - accuracy: 0.7701
Epoch 8/100
226/226 [==============================] - 0s 881us/step - loss: 0.4741 - accuracy: 0.7713
Epoch 9/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4719 - accuracy: 0.7764
Epoch 10/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4691 - 

### 3. SMOTE

In [9]:
X = df2.drop('Churn', axis='columns')
y = df2['Churn']

smote = SMOTE(sampling_strategy='minority')
(Xs, ys) = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.3)
y_pred = ANN(X_train, X_test, y_train, y_test)

Epoch 1/100
226/226 [==============================] - 3s 3ms/step - loss: 0.5265 - accuracy: 0.7536
Epoch 2/100
226/226 [==============================] - 1s 3ms/step - loss: 0.4687 - accuracy: 0.7788
Epoch 3/100
226/226 [==============================] - 1s 3ms/step - loss: 0.4599 - accuracy: 0.7791
Epoch 4/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4548 - accuracy: 0.7822
Epoch 5/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4504 - accuracy: 0.7849
Epoch 6/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4476 - accuracy: 0.7880
Epoch 7/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4443 - accuracy: 0.7890
Epoch 8/100
226/226 [==============================] - 0s 1ms/step - loss: 0.4405 - accuracy: 0.7868
Epoch 9/100
226/226 [==============================] - 0s 2ms/step - loss: 0.4382 - accuracy: 0.7911
Epoch 10/100
226/226 [==============================] - 0s 2ms/step - loss: 0.4348 - accura

### 4. Use of Ensemble with Undersampling

In [15]:
X = df2.drop('Churn', axis='columns')
y = df2['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

min_class = min(y.value_counts())

div = X_train.shape[0] // min_class
ans = np.array([.0] * y_test.shape[0])

for i in range(div):
    X_train_new = X_train[i * min_class : (i + 1) * min_class]
    y_train_new = y_train[i * min_class : (i + 1) * min_class]
    y_pred = ANN(X_train_new, X_test, y_train_new, y_test)
    ans += y_pred.reshape(y_pred.shape[0])

X_train_new = X_train[X_train.shape[0] - min_class:]
y_train_new = y_train[y_train.shape[0] - min_class:]
y_pred = ANN(X_train_new, X_test, y_train_new, y_test)
ans += y_pred.reshape(y_pred.shape[0])

y_pred_new = np.array(ans > 1, dtype=np.uint8)
print(classification_report(y_test, y_pred_new))

Epoch 1/100
59/59 [==============================] - 1s 1ms/step - loss: 0.6134 - accuracy: 0.6672
Epoch 2/100
59/59 [==============================] - 0s 1ms/step - loss: 0.5038 - accuracy: 0.7325
Epoch 3/100
59/59 [==============================] - 0s 1ms/step - loss: 0.4639 - accuracy: 0.7699
Epoch 4/100
59/59 [==============================] - 0s 1ms/step - loss: 0.4488 - accuracy: 0.7801
Epoch 5/100
59/59 [==============================] - 0s 1ms/step - loss: 0.4427 - accuracy: 0.7806
Epoch 6/100
59/59 [==============================] - 0s 1ms/step - loss: 0.4397 - accuracy: 0.7806
Epoch 7/100
59/59 [==============================] - 0s 1ms/step - loss: 0.4371 - accuracy: 0.7865
Epoch 8/100
59/59 [==============================] - 0s 1ms/step - loss: 0.4346 - accuracy: 0.7865
Epoch 9/100
59/59 [==============================] - 0s 1ms/step - loss: 0.4341 - accuracy: 0.7929
Epoch 10/100
59/59 [==============================] - 0s 2ms/step - loss: 0.4307 - accuracy: 0.7929
Epoch 11/